In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
! pip install rasterio  -q -q -q
! pip install geopandas -q -q -q
! pip install matplotlib -q -q -q

In [18]:
!pip install scikit-image

In [5]:
import sys
sys.path.append('../src')
from classes.data.satellite_image import SatelliteImage
from utils.utils import *
from utils.plot_utils import *
from utils.image_utils import *
import utils.mappings as mapps

In [6]:
import yaml
import re
import s3fs
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as im
import re
from pyproj import Transformer
from datetime import date
from scipy.ndimage import label
import time
from tqdm import tqdm
import os

In [7]:
update_storage_access()

In [8]:
environment = get_environment()

root_path = get_root_path()

bucket = environment["bucket"]

#2022
#guyane 2022
path_s3_pleiades_data_2022_guyane = environment["sources"]["PLEIADES"][2022]["guyane"]
path_local_pleiades_data_2022_guyane = environment["local-path"]["PLEIADES"][2022]["guyane"]

#martinique 2022
path_s3_pleiades_data_2022_martinique = environment["sources"]["PLEIADES"][2022]["martinique"]
path_local_pleiades_data_2022_martinique = environment["local-path"]["PLEIADES"][2022]["martinique"]
          
#guadeloupe 2022
path_s3_pleiades_data_2022_guadeloupe = environment["sources"]["PLEIADES"][2022]["guadeloupe"]
path_local_pleiades_data_2022_guadeloupe = environment["local-path"]["PLEIADES"][2022]["guadeloupe"]

#mayotte 2022
path_s3_pleiades_data_2022_mayotte = environment["sources"]["PLEIADES"][2022]["976"]
path_local_pleiades_data_2022_mayotte = environment["local-path"]["PLEIADES"][2022]["976"]

#2020
#mayotte 2020
path_s3_pleiades_data_2020_mayotte = environment["sources"]["PLEIADES"][2020]["976"]
path_local_pleiades_data_2020_mayotte = environment["local-path"]["PLEIADES"][2020]["976"]

#2019
#mayotte 2019
path_s3_pleiades_data_2019_mayotte = environment["sources"]["PLEIADES"][2019]["976"]
path_local_pleiades_data_2019_mayotte = environment["local-path"]["PLEIADES"][2019]["976"]

#2018
#reunion 2018
path_s3_pleiades_data_2018_reunion = environment["sources"]["PLEIADES"][2018]["reunion"]
path_local_pleiades_data_2018_reunion = environment["local-path"]["PLEIADES"][2018]["reunion"]

#2017
#martinique 2017
path_s3_pleiades_data_2017_martinique = environment["sources"]["PLEIADES"][2017]["martinique"]
path_local_pleiades_data_2017_martinique = environment["local-path"]["PLEIADES"][2017]["martinique"]

#mayotte 2017
path_s3_pleiades_data_2017_mayotte = environment["sources"]["PLEIADES"][2017]["976"]
path_local_pleiades_data_2017_mayotte = environment["local-path"]["PLEIADES"][2017]["976"]

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

In [9]:
environment = get_environment()
root_path = get_root_path()
bucket = environment["bucket"]

#mayotte 2022
path_s3_pleiades_data_2022_mayotte = environment["sources"]["PLEIADES"][2022]["976"]
path_local_pleiades_data_2022_mayotte = environment["local-path"]["PLEIADES"][2022]["976"]

#2020
#mayotte 2020
path_s3_pleiades_data_2020_mayotte = environment["sources"]["PLEIADES"][2020]["976"]
path_local_pleiades_data_2020_mayotte = environment["local-path"]["PLEIADES"][2020]["976"]

#2019
#mayotte 2019
path_s3_pleiades_data_2019_mayotte = environment["sources"]["PLEIADES"][2019]["976"]
path_local_pleiades_data_2019_mayotte = environment["local-path"]["PLEIADES"][2019]["976"]

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

In [10]:
# DL PLEIADE mayotte 2020
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2020_mayotte}",
        lpath=f"../{path_local_pleiades_data_2020_mayotte}",
        recursive=True)

In [ ]:
# DL PLEIADE mayotte 2017
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2017_mayotte}",
        lpath=f"../{path_local_pleiades_data_2017_mayotte}",
        recursive=True)

In [9]:
# DL PLEIADE martinique 2022 utilisé !
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2022_martinique}",
        lpath=f"../{path_local_pleiades_data_2022_martinique}",
        recursive=True)

In [57]:
# DL PLEIADE martinique 2017 utilisé !
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2017_martinique}",
        lpath=f"../{path_local_pleiades_data_2017_martinique}",
        recursive=True)

In [6]:
# DL PLEIADE guyane 2022 utilisé !
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2022_guyane}",
        lpath=f"../{path_local_pleiades_data_2022_guyane}",
        recursive=True)

En martinique :
(14.635338, -61.038345) plateau médico-technique construit en 2018 (côté droit du triangle). 
Essayons de retrouver l'image qui le contient.

In [210]:
#Nouvelle image 1
filename_1 = find_image_of_point([14.635338, -61.038345], '../data/PLEIADES/2022/MARTINIQUE', coord_gps = True)
date_1 = date.fromisoformat('2022-01-01')

In [129]:
image_1 = SatelliteImage.from_raster(
        filename_1,
        date = date_1, 
        n_bands = 3,
        dep = "972"
    )
image_1.normalize()

In [130]:
image_1.plot([0,1,2]) 

In [97]:
#image 2
filename_2 = find_image_different_years(2017, None, filename_1)
date_2 = date.fromisoformat('2017-01-01')

In [98]:
image_2 = SatelliteImage.from_raster(
        filename_2,
        date = date_2, 
        n_bands = 3,
        dep = "972"
    )
image_2.normalize()

In [89]:
image_2.plot([0,1,2])

On a bien un changement à droite du triangle. Il ne reste plus qu'a avoir la bande infrarouge pour faire les masques puis les différences.

On va chercher en guyane : (-12.755601, 45.229945) coordonnées d'un quartier

In [31]:
filename_5 = find_image_of_point([-12.755601, 45.229945], '../data/PLEIADES/2020/MAYOTTE', coord_gps = True)
date_5 = date.fromisoformat('2020-01-01')
print(filename_5)

In [32]:
image_5 = SatelliteImage.from_raster(
        filename_5,
        date = date_5, 
        n_bands = 3,
        dep = "976"
    )

In [33]:
image_5.plot([0,1,2])
image_5 = SatelliteImage.from_raster(
        filename_5,
        date = date_5, 
        n_bands = 3,
        dep = "976"
    )

In [39]:
from utils.gestion_label_studio import create_mask_from_label_studio_export

create_mask_from_label_studio_export(
            num_task = "3",
            name_output = "mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits"
        )

mask  = np.load("../export/mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits.npy")
plt.imshow(np.transpose(mask), cmap="gray", interpolation="nearest")

np.save("../export/mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits.npy",np.transpose(mask))

mc cp s3/projet-slums-detection/Donnees/PLEIADES/MAYOTTE/2020/MOSA/ORT_2020052526670967_0524_8590_U38S_8Bits.jp2 s3/projet-slums-detection/donnees-test/segmentation/images/mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits.jp2
mc cp export/mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits.npy s3/projet-slums-detection/donnees-test/segmentation/masks/mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits.npy


In [148]:
filename_6 = find_image_different_years(2017, image_5)
date_6 = date.fromisoformat('2020-01-01')

In [149]:
image_6 = SatelliteImage.from_raster(
        filename_6,
        date = date_6, 
        n_bands = 3,
        dep = "976"
    )
image_6.normalize()

In [150]:
image_6.plot([0,1,2])

Dans la moitié haute de l'image à droite, il y a plusieurs bâtiments qui ont été construits.

Pas de RIL pour ces deux années en mayotte ...

In [16]:
filename_7 = find_image_of_point([-12.753436, 45.219888], '../data/PLEIADES/2020/MAYOTTE', coord_gps = True)
date_7 = date.fromisoformat('2020-01-01')
print(filename_7)

In [17]:
image_7 = SatelliteImage.from_raster(
        filename_7,
        date = date_7, 
        n_bands = 3,
        dep = "976"
    )

In [18]:
image_7.plot([0,1,2])
image_7 = SatelliteImage.from_raster(
        filename_7,
        date = date_7, 
        n_bands = 3,
        dep = "976"
    )

In [30]:
from utils.gestion_label_studio import create_mask_from_label_studio_export

create_mask_from_label_studio_export(
            num_task = "1",
            name_output = "mayotte-ORT_2020052526670967_0523_8591_U38S_8Bit"
        )


In [42]:
mask  = np.load("../donnees-test/segmentation/masks/mayotte-ORT_2020052526670967_0523_8591_U38S_8Bit.npy")

plt.imshow(np.transpose(mask), cmap="gray", interpolation="nearest")

#np.save("../export/mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits.npy",np.transpose(mask))

In [46]:
image = np.transpose(mask).copy()

# Étape 2 : Dimensions de l'image
height = len(image)
width = len(image[0])

# Étape 3 : Création du nouveau tableau
rotated_image = [[0] * height for _ in range(width)]

# Étape 4 : Rotation de l'image
for i in range(height):
    for j in range(width):
        rotated_image[width - 1 - j][i] = image[i][j]

In [47]:
plt.imshow(rotated_image, cmap="gray", interpolation="nearest")

In [48]:
np.save("../donnees-test/segmentation/masks/mayotte-ORT_2020052526670967_0523_8591_U38S_8Bit.npy",rotated_image)

export MC_HOST_s3=https://5BYL3V2CNPM3E6ZKP233:2XseSTXTVbP38zJWyQ+ccPSAtFydziWEMKcexwuS:eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiI1QllMM1YyQ05QTTNFNlpLUDIzMyIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNjgzMjg5MDkyLCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6InJheWEuYmVyb3ZhQGVsZXZlLmVuc2FpLmZyIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImV4cCI6MTY4MzgxMDk1MSwiZmFtaWx5X25hbWUiOiJCRVJPVkEiLCJnaXZlbl9uYW1lIjoiUmF5YSIsImdyb3VwcyI6WyJmdW5hdGhvbiIsInNsdW1zLWRldGVjdGlvbiJdLCJpYXQiOjE2ODMyODkwOTQsImlzcyI6Imh0dHBzOi8vYXV0aC5sYWIuc3NwY2xvdWQuZnIvYXV0aC9yZWFsbXMvc3NwY2xvdWQiLCJqdGkiOiI2MjcyNTg2Ni02OTRlLTQzNWQtOWU2Ni04NmRlNmE5ZDFkZjkiLCJsb2NhbGUiOiJlbiIsIm5hbWUiOiJSYXlhIEJFUk9WQSIsIm5vbmNlIjoiNTExYWU3YjUtY2YzNi00ODEyLWE2Y2MtZDIyMDIxMTUzZjQwIiwicG9saWN5Ijoic3Rzb25seSIsInByZWZlcnJlZF91c2VybmFtZSI6ImlkMTc1MyIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoib3BlbmlkIHByb2ZpbGUgZ3JvdXBzIGVtYWlsIiwic2Vzc2lvblBvbGljeSI6ImV5SldaWEp6YVc5dUlqb2lNakF4TWkweE1DMHhOeUlzSWxOMFlYUmxiV1Z1ZENJNlczc2lSV1ptWldOMElqb2lRV3hzYjNjaUxDSkJZM1JwYjI0aU9sc2ljek02S2lKZExDSlNaWE52ZFhKalpTSTZXeUpoY200NllYZHpPbk16T2pvNmNISnZhbVYwTFhOc2RXMXpMV1JsZEdWamRHbHZiaUlzSW1GeWJqcGhkM002Y3pNNk9qcHdjbTlxWlhRdGMyeDFiWE10WkdWMFpXTjBhVzl1THlvaVhYMHNleUpGWm1abFkzUWlPaUpCYkd4dmR5SXNJa0ZqZEdsdmJpSTZXeUp6TXpwTWFYTjBRblZqYTJWMElsMHNJbEpsYzI5MWNtTmxJanBiSW1GeWJqcGhkM002Y3pNNk9qb3FJbDBzSWtOdmJtUnBkR2x2YmlJNmV5SlRkSEpwYm1kTWFXdGxJanA3SW5Nek9uQnlaV1pwZUNJNkltUnBabVoxYzJsdmJpOHFJbjE5ZlN4N0lrVm1abVZqZENJNklrRnNiRzkzSWl3aVFXTjBhVzl1SWpwYkluTXpPa2RsZEU5aWFtVmpkQ0pkTENKU1pYTnZkWEpqWlNJNld5SmhjbTQ2WVhkek9uTXpPam82S2k5a2FXWm1kWE5wYjI0dktpSmRmVjE5Iiwic2Vzc2lvbl9zdGF0ZSI6ImQ5NTg2MzBiLTBjOWMtNGE0ZC05ZDU1LTE5M2YxY2NiNjNlMiIsInNpZCI6ImQ5NTg2MzBiLTBjOWMtNGE0ZC05ZDU1LTE5M2YxY2NiNjNlMiIsInN1YiI6ImRiNWM5MTg3LWUwNjItNDIyMy04N2UwLTk2YTc5ODYzMDZlOSIsInR5cCI6IkJlYXJlciJ9.mjfzD-pGSyEIx3i6vJP9lCW-rqhd9EaZsRbnCR39DJtQwdUcGn-Sk6KYKmMf4Q3YkeAT8mc7Rx-xtu0eXPfFhA@minio.lab.sspcloud.fr

mc cp s3/projet-slums-detection/Donnees/PLEIADES/MAYOTTE/2020/MOSA/ORT_2020052526670967_0523_8591_U38S_8Bits.jp2 s3/projet-slums-detection/donnees-test/segmentation/images/mayotte-ORT_2020052526670967_0523_8591_U38S_8Bits.jp2

mc cp export/mayotte-ORT_2020052526670967_0523_8591_U38S_8Bit.npy s3/projet-slums-detection/donnees-test/segmentation/masks/mayotte-ORT_2020052526670967_0523_8591_U38S_8Bit.npy


In [16]:
from PIL import Image
import numpy as np
import io

# Créer un tableau avec des données d'exemple
data = image_7.array
data = (data * 255).astype(np.uint8)
data = data.transpose(2,1,0)


# Convertir le tableau en image
image = Image.fromarray(data)

# Sauvegarder l'image en tant que fichier PNG
image.save("image.png")

# Enregistrer l'image en tant qu'objet bytes pour téléchargement
img_bytes = io.BytesIO()
image.save(img_bytes, format='PNG')
img_bytes.seek(0)

# Envoyer l'image pour téléchargement
from IPython.display import FileLink
display(FileLink("image.png"))

In [16]:
filename_8 = find_image_different_years(2017, image_7)
date_8 = date.fromisoformat('2020-01-01')

In [17]:
image_8 = SatelliteImage.from_raster(
        filename_7,
        date = date_7, 
        n_bands = 3,
        dep = "976"
    )
image_8.normalize()

In [18]:
image_8.plot([0,1,2])